### **Propósito**
Calcular la **Huella Hídrica Verde (agua de lluvia)** y la **Huella Hídrica Azul (agua de riego)** para un cultivo de **trigo**, a partir de datos meteorológicos diarios proporcionados por el sistema **REMAS**.

---

### **Contexto**
- El cultivo elegido es **trigo**, ya que contamos con:
  - Su **periodo de crecimiento definido**.
  - Parámetros **Kc** para cada fase del desarrollo.
  - Un valor estimado de **rendimiento agrícola (ton/ha)** obtenido del **SIAP**.
- El municipio y la estación meteorológica pueden ajustarse cambiando el nombre de la estación en el código.  
  *(Actualmente, solo se considera la estación meteorológica como filtro).*

---

### **Archivos Utilizados**
Actualmente se utilizan **dos archivos Excel**:

1. `REMAS_Etchojoa_Cajeme_2022-2023.xlsx`  
   - Contiene datos diarios de **precipitación, temperatura, humedad, radiación**, etc., por estación meteorológica.

2. `ETo_REMAS_Etchojoa_Cajeme_2022-2023.xlsx`  
   - Contiene valores diarios de **Evapotranspiración de referencia (ETo)**.

> **Importante:** en el código se debe especificar correctamente la ruta donde se encuentran los archivos. Asegúrate de ajustarla según la ubicación real de tus datos.

### **Ubicación de Estaciones Meteorológicas**
Para consultar la **ubicación geográfica** de cada estación meteorológica, visita:  
🔗 [https://remas.siafeson.org.mx/index.php/Mapas](https://remas.siafeson.org.mx/index.php/Mapas)

### **Propósito**
Calcular la **Huella Hídrica Verde (agua de lluvia)** y la **Huella Hídrica Azul (agua de riego)** para un cultivo de **trigo**, a partir de datos meteorológicos diarios proporcionados por el sistema **REMAS**.

---

### **Contexto**
- El cultivo elegido es **trigo**, ya que contamos con:
  - Su **periodo de crecimiento definido**.
  - Parámetros **Kc** para cada fase del desarrollo.
  - Un valor estimado de **rendimiento agrícola (ton/ha)** obtenido del **SIAP**.
- El municipio y la estación meteorológica exactos pueden ajustarse cambiando el nombre de la estación en el código.  
  *(Actualmente, solo se considera la estación meteorológica como filtro).*

---

### **Archivos Utilizados**
Actualmente se utilizan **dos archivos Excel**:

1. `REMAS_Etchojoa_Cajeme_2022-2023.xlsx`  
   - Contiene datos diarios de **precipitación, temperatura, humedad, radiación**, etc., por estación meteorológica.

2. `ETo_REMAS_Etchojoa_Cajeme_2022-2023.xlsx`  
   - Contiene valores diarios de **Evapotranspiración de referencia (ETo)**.

> **Importante:** en el código se debe especificar correctamente la ruta donde se encuentran los archivos. Asegúrate de ajustarla según la ubicación real de tus datos.

---

### **Ubicación de Estaciones Meteorológicas**
Para consultar la **ubicación geográfica** de cada estación meteorológica, visita:  
🔗 [https://remas.siafeson.org.mx/index.php/Mapas](https://remas.siafeson.org.mx/index.php/Mapas)


In [21]:
import pandas as pd
from datetime import datetime, timedelta

In [22]:
# Cargar el dataset principal con los datos meteorológicos de las estaciones
df_meteorologico = pd.read_excel('./Datos/REMAS_Etchojoa_Cajeme_2022-2023.xlsx')

In [23]:
# Cargar el dataset con los valores de ETo
df_eto = pd.read_excel('./Datos/ETo_REMAS_Etchojoa_Cajeme_2022-2023.xlsx')

In [24]:
# Filtrar por el municipio y la estación deseada
municipio_seleccionado = "Cajeme"  # Cambia el municipio aquí (No se utiliza aún)
estacion_seleccionada = "BLOCK 910 (CIANO)"  # Cambia el nombre de la estación específica aquí, revisar mapa de remas para elegir cualquiere de los municipios de Etchojoa o Cajeme

df_meteorologico = df_meteorologico[
    (df_meteorologico['Municipio'] == municipio_seleccionado) & 
    (df_meteorologico['Estación'] == estacion_seleccionada)
]

In [25]:
# Renombrar las columnas en df_eto para facilitar la unión
df_eto.rename(columns={'REFERENCIA': 'Estación', 'FECHA': 'Fecha'}, inplace=True)

In [26]:
# Asegurarnos de que ambas columnas de fecha están en el mismo formato
df_meteorologico['Fecha'] = pd.to_datetime(df_meteorologico['Fecha'], format='%m/%d/%Y')
df_eto['Fecha'] = pd.to_datetime(df_eto['Fecha'], format='%m/%d/%Y')

In [27]:
# Filtrar el DataFrame de ETo por la misma estación
df_eto = df_eto[df_eto['Estación'] == estacion_seleccionada]

In [28]:
# Unir ambos DataFrames en función de la estación (Estación) y la fecha (Fecha)
df = pd.merge(df_meteorologico, df_eto[['Estación', 'Fecha', 'ETO']], on=['Estación', 'Fecha'], how='inner')

In [29]:
# Definir los parámetros del cultivo (trigo)
pcc = 140  # Periodo de crecimiento del cultivo en días
kc_values = {
    'inicio': 0.15,  # Kc inicial
    'medio': 1.15,   # Kc medio
    'final': 0.50    # Kc final
}
rendimiento = 7.37  # Rendimiento del cultivo en ton/ha Obtenido del SIAP

In [30]:
# Definir la fecha de inicio en el código
fecha_inicio_str = "11/30/2022"  # Cambia la fecha de inicio según el periodo deseado
fecha_inicio = datetime.strptime(fecha_inicio_str, '%m/%d/%Y')

# Verificar si hay suficientes días para el periodo de cultivo
fecha_fin = fecha_inicio + timedelta(days=pcc - 1)
ultima_fecha = df['Fecha'].max()

In [33]:
if fecha_fin > ultima_fecha:
    print(f"No hay suficientes datos para un periodo de {pcc} días desde la fecha de inicio {fecha_inicio_str}.")
else:
    # Filtrar el DataFrame para el periodo de 140 días desde la fecha de inicio y hacer una copia para evitar warnings
    df_periodo = df[(df['Fecha'] >= fecha_inicio) & (df['Fecha'] <= fecha_fin)].copy()

In [36]:
    # Reiniciar el índice en el DataFrame filtrado, si es necesario
    df_periodo = df_periodo.reset_index(drop=True)

    # Crear y asignar la columna 'Kc' como float para evitar problemas de tipo de datos
    df_periodo['Kc'] = 0.0
    df_periodo.loc[df_periodo.index < 45, 'Kc'] = kc_values['inicio']
    df_periodo.loc[(df_periodo.index >= 45) & (df_periodo.index < 95), 'Kc'] = kc_values['medio']
    df_periodo.loc[df_periodo.index >= 95, 'Kc'] = kc_values['final']

In [37]:
    # Calcular ETc (Evapotranspiración de cultivo)
    df_periodo['ETc'] = df_periodo['Kc'] * df_periodo['ETO']

In [38]:
    print(df_periodo[['ETc', 'Kc', 'ETO']])

        ETc    Kc   ETO
0    0.6255  0.15  4.17
1    0.7560  0.15  5.04
2    0.9090  0.15  6.06
3    0.6900  0.15  4.60
4    0.6135  0.15  4.09
5    0.2955  0.15  1.97
6    0.2025  0.15  1.35
7    0.3465  0.15  2.31
8    0.6270  0.15  4.18
9    0.6225  0.15  4.15
10   0.6285  0.15  4.19
11   0.5895  0.15  3.93
12   0.6420  0.15  4.28
13   0.5160  0.15  3.44
14   0.6915  0.15  4.61
15   0.6690  0.15  4.46
16   0.4485  0.15  2.99
17   0.5895  0.15  3.93
18   0.5685  0.15  3.79
19   0.6975  0.15  4.65
20   0.7155  0.15  4.77
21   0.5940  0.15  3.96
22   0.6195  0.15  4.13
23   0.6180  0.15  4.12
24   0.6570  0.15  4.38
25   0.7785  0.15  5.19
26   0.6870  0.15  4.58
27   0.6945  0.15  4.63
28   0.6450  0.15  4.30
29   0.2400  0.15  1.60
30   0.5145  0.15  3.43
31   0.5700  0.15  3.80
32   0.4530  0.15  3.02
33   0.4545  0.15  3.03
34   0.5190  0.15  3.46
35   0.6690  0.15  4.46
36   0.7755  0.15  5.17
37   0.7485  0.15  4.99
38   0.6075  0.15  4.05
39   0.5925  0.15  3.95
40   0.6510  0.1

In [39]:

    # Definir la función para calcular Precipitación efectiva
    def calcular_precip_efectiva(precipitacion_total):
        if precipitacion_total < 250:
            # Cálculo de la precipitación efectiva cuando es menor a 250 mm
            precip_efectiva = precipitacion_total * (125 - 0.2 * precipitacion_total) / 125
        else:
            # Cálculo de la precipitación efectiva cuando es mayor o igual a 250 mm
            precip_efectiva = 125 + 0.1 * precipitacion_total
        return precip_efectiva

    # Calcular Precipitación efectiva para cada registro
    df_periodo['Precipitación efectiva'] = df_periodo['Precipitación (mm)'].apply(calcular_precip_efectiva)

    # Calcular ETverde (Evapotranspiración de agua verde)
    df_periodo['ETverde'] = df_periodo[['ETc', 'Precipitación efectiva']].min(axis=1)

    # Calcular ETazul (Evapotranspiración de agua azul)
    df_periodo['ETazul'] = (df_periodo['ETc'] - df_periodo['Precipitación efectiva']).apply(lambda x: max(0, x))

    # Calcular el uso de agua verde y agua azul total en el periodo de 140 días
    uso_agua_verde = 10 * df_periodo['ETverde'].sum()  # m³/ha
    uso_agua_azul = 10 * df_periodo['ETazul'].sum()    # m³/ha

    # Calcular la Huella Hídrica en m³/ton para agua verde y agua azul
    hh_proc_verde = uso_agua_verde / rendimiento
    hh_proc_azul = uso_agua_azul / rendimiento

    # Mostrar los resultados
    print(f"Uso de Agua Verde en Cultivo para los {pcc} días desde {fecha_inicio_str}: {uso_agua_verde} m³/ha")
    print(f"Uso de Agua Azul en Cultivo para los {pcc} días desde {fecha_inicio_str}: {uso_agua_azul} m³/ha")
    print(f"Huella Hídrica de Agua Verde (HHproc, verde): {hh_proc_verde} m³/ton")
    print(f"Huella Hídrica de Agua Azul (HHproc, azul): {hh_proc_azul} m³/ton")

Uso de Agua Verde en Cultivo para los 140 días desde 11/30/2022: 138.19548 m³/ha
Uso de Agua Azul en Cultivo para los 140 días desde 11/30/2022: 3764.9345199999993 m³/ha
Huella Hídrica de Agua Verde (HHproc, verde): 18.75108276797829 m³/ton
Huella Hídrica de Agua Azul (HHproc, azul): 510.84593215739477 m³/ton
